# Short introduction to Julia

- Oscar van der Heide
- Phd at UMC Utrecht working on MRI

- I'm **not** a computer scientist

- The goal of this presentation will **not** be to convert you to Julia

# 1. Why C/C++/Fortran are fast

- C/C++/Fortran are compiled languages

- At "compile time", a compiler translates your human-readable code into machine instructions

- At "runtime", computer executes the generated machine instructions

- Not only does the compiler translate your code, it also performs many optimizations

In [ ]:
### Pseudo C/C++/Fortran code
function f(x)
   
    s = 0;
    
    for i = 1:length(x)
        y = 2*x[i] + 3*x[i]^3
        s += y
    end
    
    return s
end

- The compiler will not compile such code. Why?

- Functions like "+" and "\*" can mean different things to a computer: adding 64 bit floats, 32 bit integers or booleans are all different

- Each of the different implementations of "+" require different machine instructions to be executed

- Since the types are not specified in the program, the compiler will throw an error: it does not know what machine instructions to generate


### Pseudo C/C++/Fortran code

In [ ]:
function f("Array of Float64's" x)
   
    "Float64" s = 0;
    
    for "Int64" i = 1:length(x)
        "Float64" y = 2*x[i] + 3*x[i]^3
        s += y
    end
    
    return s
end

- Now all types are known at "compile time" because we provided type annotations

- Compiler now knows exactly what machine instructions to generate for all the {+,-,*,^,...}'s

- Compiler will also recognize that the same operation is performed many times in a loop and will perform optimizations (pipelining, SIMD vector instructions, function inlining)

- The result will be *fast code*

### But development in C is not so fast ...

- Because of the compilation required, you can't run code in an interactive session: you write your whole program first, compile it and then run it

- Because of the required type annotations, you'll end up having to write a versions of `f` for `f_array_float64`, `f_array_float32`, `f_array_int64`, `f_array_int32`

- But also maybe `f_array_complex_float64`, `f_array_complex_float32`, `f_array_complex_int64`, `f_array_complex_int32`


- And also maybe something like `f_array_of_matrices_of_float64s`, because all the operations involved can be applied on matrices as well

- In other words, your code will not be *generic*

# 2. Why Python/Matlab/R are slow

- Python/Matlab/R are interpreted languages

- No separate compile time and runtime

- Instead, when you want to execute code, an "interpreter" parses and executes it *line-by-line*, *statement-by-statement*

- This allows for interactive sessions

- The interpreter will also take care of all the types for you so it easy to write (type) generic code 

### But there's a price to pay in terms of performance:


In [ ]:
# Pseudo Python/Matlab/R code

function f(x)
   
    s = 0;
    
    for i = 1:length(x)
        y = 2*x[i] + 3*x[i]^3 
        s += y
    end
    
    return s
end

The interpreter basically sees this as a list of separate statements that, at runtime, it will go through one-by-one:

In [ ]:
s = 0;

tmp1 = 2*x[1]
tmp2 = x[1]^3
tmp3 = 3*tmp2  
tmp4 = tmp2 + tmp3
s += tmp4
...
tmp101 = 2*x[100]
tmp102 = x[100]^3
tmp103 = 3*tmp102
tmp104 = tmp102 + tmp103
s += tmp104
...

### At each statement the interpreter: 

1. checks the types of the input arguments to the function (e.g. +,-,*,^,...)

2. retrieves a list of known implementations for the given function from memory   

3. selects the appropriate method for the current types 

4. performs the computation


The combination of the first three things is called "dynamic dispatch" or "runtime type checking". It is more time consuming than performing the actual computations (4).

In addition, because the interpreter only has a "local" view of the code, it cannot optimize your code in a way that a compiler (with it's "global" view) can. 
           

### Vectorization

- This is why you are told to not write for loops in Python/Matlab/R
- Instead, you are told to *vectorize* everything

In [ ]:
f(x) = sum(2*x + 3*x^3)

- When vectorizing code, under the hood calls are made to compiled library functions. 
- This way you can get C/C++/Fortran like performance from an interpreted language! 

# 3. Why vectorized code may not be optimal

- First of all, not all code can be properly vectorized ... 

- Secondly, and this is what we will focus on, vectorized code may be far from optimal

- Let's look at an example (taken from https://julialang.org/blog/2017/01/moredots)

- Suppose we have a function `f(y) = 3y^2 + 5y + 2` that evaluates a polynomial

- And suppose we want to want to evaluate `f(2x^2 + 6x^3 - sqrt(x))` for a whole array `X`, storing the result in-place in `X`.

In [ ]:
# Vectorized code

N = 10^7
X = rand(N,1)

# Evaluate input to polynomial f
Y = 2*X.^2 + 6*X.^3 - sqrt(X)

# Evaluate polynomial f
X = 3*Y.^2 + 5*Y + 2*ones(size(Y))

There is no optimized C code for the operation as a whole, only for the individual components. Therefore the vectorized code will effectively do something like:

In [ ]:
# Y = 2*X.^2 + 6*X.^3 - sqrt(X)
# X = 3*Y.^2 + 5*Y + 2*ones(size(Y))

# Evaluate input to polynomial f
tmp1 = X.^2 
tmp2 = 2*tmp1
tmp3 = X.^3
tmp4 = 6 * tmp3
tmp5 = tmp2 + tmp4
tmp6 = sqrt(X)
tmp7 = tmp5 - tmp6
# Evaluate polynomial f
tmp8 = tmp7.^2
tmp9 = 3*tmp8
tmp10 = 5*tmp7
tmp11 = tmp9 + tmp10
tmp12 = tmp11 + 2*ones(size(X)) 
X = tmp12

Two problems with this vectorized code:

1. Many unnecessary memory allocations. Will hurt performance especially if X is either very small or very large.

2. In each line there is a new loop in which memory (RAM) is accessed and only a single operation (+,-, ^,sqrt,...) is performed. Memory access in general is expensive and it is better to retrieve each `X[i]` only once from memory and do as much scalar operations (`f(2*x^2 + 6*x^3 - sqrt(x)`) at once.


You can solve these problems two problems with loops. But loops are slow in interpreted languages. So to get optimal performance (actual speedup compared to vectorized code will be shown later) you would have to write C/C++/Fortran code.

Or ... use Julia :)

# 4. Julia

- Julia is relatively new programming language: development started in 2009 at MIT, first release 2012, stable 1.0 release in 2018

- It tries to bring you the best of both worlds: write type generic code like in an interpreted language with performance similar to compiled languages. 

- This is achieved through a combination of: 

    1. Just-in-time (JIT) compilation
    2. Type inference
    3. Multiple dispatch (not discussed)

### 4.1 Just-in-time (JIT) compilation

In [1]:
function f(X)
   
    s = zero(eltype(X));
    
    for x ∈ X
        s += 2x + 3x^3
    end
    
    return s
end

X = rand(10^6)
@time f(X); 

  0.015172 seconds (40.74 k allocations: 2.216 MiB)


Why all these allocations?! It's because the first time a function is called, it gets compiled

Let's call the function again 

In [2]:
@time f(X);

  0.002332 seconds (5 allocations: 176 bytes)


Ahh that's better. Note that the 5 allocations are related to timing in global scope, there's a better way to benchmark functions as will be shown later

### 4.2 Type inference

- Notice that we did not specify the types of the variables in our function `f`. 

- How can the compiler compile if it doesn't know the types?

- The compiler performs *type inference*: given the types of the input arguments to the function, it will try to deduce the types of all other variables

In [5]:
@code_warntype f(X)

Variables
  #self#::Core.Compiler.Const(f, false)
  X::Array{Float64,1}
  s::Float64
  @_4::Union{Nothing, Tuple{Float64,Int64}}
  x::Float64

Body::Float64
1 ─ %1  = Main.eltype(X)::Core.Compiler.Const(Float64, false)
│         (s = Main.zero(%1))
│   %3  = X::Array{Float64,1}
│         (@_4 = Base.iterate(%3))
│   %5  = (@_4 === nothing)::Bool
│   %6  = Base.not_int(%5)::Bool
└──       goto #4 if not %6
2 ┄ %8  = @_4::Tuple{Float64,Int64}::Tuple{Float64,Int64}
│         (x = Core.getfield(%8, 1))
│   %10 = Core.getfield(%8, 2)::Int64
│   %11 = s::Float64
│   %12 = (2 * x)::Float64
│   %13 = x::Float64
│   %14 = Core.apply_type(Base.Val, 3)::Core.Compiler.Const(Val{3}, false)
│   %15 = (%14)()::Core.Compiler.Const(Val{3}(), false)
│   %16 = Base.literal_pow(Main.:^, %13, %15)::Float64
│   %17 = (3 * %16)::Float64
│   %18 = (%12 + %17)::Float64
│         (s = %11 + %18)
│         (@_4 = Base.iterate(%3, %10))
│   %21 = (@_4 === nothing)::Bool
│   %22 = Base.not_int(%21)::Bool
└──      

### 4.2 Type inference

- If the type inference is succesful, the compiler will be able to generate efficient machine instructions:

In [1]:
@code_native f(X)

UndefVarError: UndefVarError: X not defined

- So far we only called our function f with an array of `Float64`s as input

- What if we call it with some other input, say, an array of `Int32`s?

- The compiler sees that a function is called with *new* types of input variables and it will again start a compilation procedure:

In [7]:
X2 = rand(Int32, 10^6)
@time f(X2);

  0.016027 seconds (18.66 k allocations: 996.384 KiB)


Let's call the function again

In [8]:
@time f(X2);

  0.003603 seconds (5 allocations: 176 bytes)


If we now look at the generated machine instructions, they will be different:

In [9]:
@code_native f(X2);

	.section	__TEXT,__text,regular,pure_instructions
; ┌ @ In[1]:5 within `f'
; │┌ @ array.jl:704 within `iterate' @ array.jl:704
; ││┌ @ In[1]:3 within `length'
	pushq	%rax
	movq	8(%rsi), %r10
; ││└
	testq	%r10, %r10
	jle	L151
; ││┌ @ array.jl:728 within `getindex'
	movq	(%rsi), %r9
	movb	$1, %dl
	xorl	%ecx, %ecx
	movl	$1, %r11d
	movabsq	$-4294967296, %r8       ## imm = 0xFFFFFFFF00000000
	xorl	%esi, %esi
	movslq	-4(%r9,%r11,4), %rax
	andb	$3, %dl
; │└└
; │ @ In[1]:6 within `f'
	cmpb	$1, %dl
	jne	L89
	jmp	L112
	nopw	%cs:(%rax,%rax)
; │ @ In[1]:5 within `f'
L64:
	movq	%rsi, %rcx
	andq	%r8, %rcx
; │ @ In[1]:6 within `f'
	addq	$1, %r11
	movb	$2, %dl
; │ @ In[1]:5 within `f'
; │┌ @ array.jl:704 within `iterate' @ array.jl:704
; ││┌ @ array.jl:728 within `getindex'
	movslq	-4(%r9,%r11,4), %rax
	andb	$3, %dl
; │└└
; │ @ In[1]:6 within `f'
	cmpb	$1, %dl
	je	L112
L89:
	cmpb	$2, %dl
	jne	L163
	movl	%esi, %edx
	orq	%rcx, %rdx
	jmp	L115
	nopw	%cs:(%rax,%rax)
; │┌ @ int.jl:872 within `+'
; ││┌ @ int

Two *specializations* of the function `f` have now been compiled and stored in memory.

But does our code really run as fast as, say, C code? Let's compare:

In [10]:
# Julia code
# using Pkg
# Pkg.add("Libdl")
using Libdl
C_code = """
#include <stddef.h>
#include <math.h>
double f_c(size_t n, double *X) 
{
    double s = 0.0;
    for (size_t i = 0; i < n; ++i) 
    {
        double x = X[i];
        double y = 2*x + 3*x*x*x;
        s = s + y;
    }
    return s;
}
"""
# compile to a shared library by piping C_code to gcc:
# (only works if you have gcc installed)
const Clib = tempname()
open(`gcc -fPIC -O3 -xc -shared -o $(Clib * "." * Libdl.dlext) -`, "w") do f
    print(f, C_code)
end

function f_c(X::Array{Float64})
    ccall(("f_c", Clib), Nothing, (Csize_t, Ptr{Float64}), length(X), X)
    return X
end

f_c (generic function with 1 method)

In [11]:
# properly time functions

using BenchmarkTools
@benchmark f_c($X);
@benchmark f($X);

ArgumentError: ArgumentError: Package BenchmarkTools not found in current path:
- Run `import Pkg; Pkg.add("BenchmarkTools")` to install the BenchmarkTools package.


- So the Julia code is running just as fast as the C code for 64 bit floats. 

- But the Julia code is generic in the sense that it can be used for all "things" for which addition and (scalar) multiplication are defined. 

- That includes (however many bits) floats, integers, complex numbers, but also vectors and matrices and it also extends to more exotic user-defined types like dual numbers (for automatic differentiation) and numbers with uncertainties. 

- That's quite powerful. 

### What if type inference fails?

- Up to now we assumed the JIT compiler was able to deduce all types

- If we write our code in a "type unstable" fashion, the compiler will not be able to deduce the types of all variables.

- Instead of throwing an error, the compiler will generate code that performs type checking at runtime.

- That is, it falls back to the dynamic dispatch behaviour that is the default in interpreted languages.

In [13]:
s = 0.0

function f_unstable(X)
    
    global s
    for x ∈ X
        s += 2x + 3x^3
    end
    
    return s
end

f_unstable (generic function with 1 method)

In [17]:
# compile
f_unstable(X);

# time
@time f_unstable(X);

# compare against type stable version
@time f(X);

  0.033227 seconds (2.00 M allocations: 30.518 MiB, 3.18% gc time)
  0.001107 seconds (5 allocations: 176 bytes)


In [18]:
@code_warntype f_unstable(X);

Variables
  #self#::Core.Compiler.Const(f_unstable, false)
  X::Array{Float64,1}
  @_3::Union{Nothing, Tuple{Float64,Int64}}
  x::Float64

Body::Any
1 ─       nothing
│   %2  = X::Array{Float64,1}
│         (@_3 = Base.iterate(%2))
│   %4  = (@_3 === nothing)::Bool
│   %5  = Base.not_int(%4)::Bool
└──       goto #4 if not %5
2 ┄ %7  = @_3::Tuple{Float64,Int64}::Tuple{Float64,Int64}
│         (x = Core.getfield(%7, 1))
│   %9  = Core.getfield(%7, 2)::Int64
│   %10 = (2 * x)::Float64
│   %11 = x::Float64
│   %12 = Core.apply_type(Base.Val, 3)::Core.Compiler.Const(Val{3}, false)
│   %13 = (%12)()::Core.Compiler.Const(Val{3}(), false)
│   %14 = Base.literal_pow(Main.:^, %11, %13)::Float64
│   %15 = (3 * %14)::Float64
│   %16 = (%10 + %15)::Float64
│   %17 = (Main.s + %16)::Any
│         (Main.s = %17)
│         (@_3 = Base.iterate(%2, %9))
│   %20 = (@_3 === nothing)::Bool
│   %21 = Base.not_int(%20)::Bool
└──       goto #4 if not %21
3 ─       goto #2
4 ┄       return Main.s


### JIT compilation + type inference + type stable code = fast code

# 5. Loops vs vectorized code

Let's go back to the earlier example of evaluating `f(2*x^2 + 6*x^3 - sqrt(x))` on an array X, with `f(y) = 3y^2 + 5y + 2`

In [19]:
function vectorized!(X)
    Y = 2*X.^2+ 6*X.^3 - sqrt.(X)
    X = 3*Y.^2 + 5*Y + 2*ones(size(Y))
end


f(y) = 3*y^2 + 5*y + 2
y(x) = 2*x^2 + 6*x^3 - sqrt(x)

function loop!(X)
    for i in 1:length(X)
        X[i] = f(y(X[i]))
    end
end

# compile
vectorized!(rand(10))
loop!(rand(10))
  
# time
X = rand(10^7)
@benchmark vectorized!(X);
@benchmark loop!(X);

LoadError: UndefVarError: @benchmark not defined

That's quite a significant difference in performance!



But vectorized code can sometimes improve readability of your code

That's why there's "dot syntax" that allow you to write vectorized code while under the hood it gets transformed into loops

In [20]:
function fused!(X)
    @. X = f(y(X))
end

# compile 
fused!(rand(10))
# time
@time fused!(X);

  0.055088 seconds (4 allocations: 160 bytes)


# 6. Conclusion

- Interpreted languages like Python/Matlab/R are slow because of type checking at runtime

- For better performance in interpreted, you vectorize code so that compiled libraries can be called

- But, optimized libraries only have code for basic operations. If you need to do more complicated operations, vectorized code performance suffers due to unnecessary memory allocations and accessing memory multiple times

- In Julia for loops are fast (similar performance to C code) even though your code can be written in a very generic manner.

# Additional things I like about Julia

- "Multiple dispatch" type system <= perhaps the most interesting aspect of Julia


- Built-in package and environment manager

- Native support for both distributed and shared memory (see v1.3) computing

- Many high quality packages for scientific computing (differential equations, optimization, machine learning, abstract math things) 

- Most of Julia and its packages are written in Julia itself: great learning resource

- Unicode character support:


In [ ]:
∫φ²dt = Δt * φ²

# Can Matlab/Python not add simply add a JIT compiler?

- Yes ... see Cython, Numba and PyPy for Python. 


- But ... limited to specific built-in types? (Numba) or compatibility issues with existing Python ecosystem (PyPy)?

- Toy problems from this presentation can probably be dealt with, but entire scientific computing packages ... ?

- Matlab also does some sort of black box JIT compilation these days ...

# So should you all switch to Julia today?

Probably not. 

If you don't care about performance that much or rely heavily on certain Python packages (although they can be called from Julia) then maybe Julia is not worth your time.

However, Julia might be interesting if you do "scientific computing stuff" where
- performance is important; and/or
- you work on new implementations/techniques/whatever rather than being simply a user of existing software packages;

But be prepared because:
- there definitely is a learning curve,
- you will have to adjust your workflow to the language

# Some Julia Resources

- https://julialang.org
- https://discourse.julialang.org
- Anything by Chris Rackauckus (https://www.stochasticlifestyle.com)
- Anything by Stefan Karpinski (https://www.youtube.com/watch?v=kc9HwsxE1OY)
- Anything by Stephen G. Johnson (https://julialang.org/blog/2017/01/moredots)
- https://julia.quantecon.org
- Package development workflow https://www.youtube.com/watch?v=QVmU29rCjaA